In [2]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('heart_disease_predictor').getOrCreate()

In [24]:
# create a structure for data types
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)
# Then create a variable with the correct structure.
data_schema = [StructField('id',IntegerType(),False),
              StructField('age',IntegerType(),False),
              StructField('gender',StringType(),False),
              StructField('height',IntegerType(),False),
              StructField('weight',IntegerType(),False),
              StructField('ap_hi',IntegerType(),False),
              StructField('ap_lo',IntegerType(),False),
              StructField('cholesterol',StringType(),False),
              StructField('gluc',StringType(),False),
              StructField('smoke',StringType(),False),
              StructField('alco',StringType(),False),
              StructField('active',StringType(),False),
              StructField('cardio',StringType(),False)]

final_struct = StructType(fields=data_schema)
# Get two data sources and merge into one table
test_csv=spark.read.option("header", "true").csv("/home/ubuntu/722_BDAS/heart_disease_test.csv", schema=final_struct)
train_csv=spark.read.option("header", "true").csv("/home/ubuntu/722_BDAS/heart_disease_train_data.csv", schema=final_struct)
df = test_csv.unionByName(train_csv, allowMissingColumns=True)
# Show the data sample (first 10 rows), table columns and count
df.show()
print(df.columns)
print(df.count())

df.select(["id", "age","gender","height","weight", "ap_hi"]).describe().show()
df.select(["ap_lo","cholesterol","gluc","smoke","alco","active"]).describe().show()
df.select(["cardio"]).describe().show()

df.printSchema()

+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|  1|20228|     1|   156|    85|  140|   90|          3|   1|    0|   0|     1|     1|
|  2|18857|     1|   165|    64|  130|   70|          3|   1|    0|   0|     0|     1|
|  3|17623|     2|   169|    82|  150|  100|          1|   1|    0|   0|     1|     1|
|  4|17474|     1|   156|    56|  100|   60|          1|   1|    0|   0|     0|     0|
|  8|21914|     1|   151|    67|  120|   80|          2|   2|    0|   0|     0|     0|
|  9|22113|     1|   157|    93|  130|   80|          3|   1|    0|   0|     1|     0|
| 12|22584|     2|   178|    95|  130|   90|          3|   3|    0|   0|     1|     1|
| 13|17668|     1|   158|    71|  110|   70|          1|   1|    0|   0|     1|     0|
| 14|19834|     1|   164|    68|  110|   60

+-------+------------------+------------------+-------------------+------------------+------------------+------------------+
|summary|                id|               age|             gender|            height|            weight|             ap_hi|
+-------+------------------+------------------+-------------------+------------------+------------------+------------------+
|  count|             69595|             69593|              69601|             69599|             69401|             69590|
|   mean|50157.826826639845|19468.128303133937| 1.3496070458757776|164.35981838819524|  74.2093053414216|128.82417013938783|
| stddev|28823.868388475246|2466.8843359438742|0.47684927003400684| 8.211530761730874|14.400164919910226| 154.4557028689057|
|    min|                 1|             10798|                  1|                55|                10|              -150|
|    max|             99999|             23713|                  2|               250|               200|             16020|


+-------+-----------------+------------------+------------------+-------------------+-------------------+------------------+
|summary|            ap_lo|       cholesterol|              gluc|              smoke|               alco|            active|
+-------+-----------------+------------------+------------------+-------------------+-------------------+------------------+
|  count|            69597|             69601|             69601|              69601|              69601|             69601|
|   mean|96.52566920987974|1.3672504705392163|1.2266203071794946| 0.0882314909268545|0.05382106578928464|0.8037671872530567|
| stddev|186.1296497832109|0.6805562160872439|0.5725085691129262|0.28363330336721265|0.22566588208767874|0.3971495462911729|
|    min|              -70|                 1|                 1|                  0|                  0|                 0|
|    max|            11000|                 3|                 3|                  1|                  1|                 1|
